In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
import numpy as np

from matplotlib import pyplot as plt
from IPython.display import clear_output

Using TensorFlow backend.


In [ ]:
#training data from https://github.com/IdleHandsProject/makennbot
x_train = [
  [ 0, 1, 1, 0 ],  # LIGHT ON LEFT AND RIGHT
  [ 0, 1, 0, 0 ],  # LIGHT ON LEFT
  [ 1, 1, 1, 0 ],  # LIGHT ON TOP, LEFT, and RIGHT
  [ 1, 1, 0, 0 ],  # LIGHT ON TOP and LEFT
  [ 0, 0, 1, 0 ],  # LIGHT ON RIGHT
  [ 1, 0, 0, 0 ],  # LIGHT ON TOP
  [ 0, 0, 0, 0 ],  # NO LIGHT
  [ 0, 0, 0, 1 ],  # LIGHT ON BOTTOM
  [ 0, 1, 0, 1 ],  # LIGHT ON BOTTOM AND LEFT
  [ 0, 0, 1, 1 ],  # LIGHT ON BOTTOM AND RIGHT
  [ 0, 1, 1, 1 ],  # LIGHT ON BOTTOM, LEFT, and RIGHT
  [ 1, 0, 0, 1 ],  # LIGHT ON TOP AND BOTTOM
  [ 1, 1, 0, 1 ],  # LIGHT ON TOP, BOTTOM, and LEFT
  [ 1, 0, 1, 1 ],  # LIGHT ON TOP, BOTTOM, and RIGHT
  [ 1, 0, 1, 0 ],  # LIGHT ON TOP AND RIGHT
  [ 1, 1, 1, 1 ],  # LIGHT ON ALL
]

y_train = [
  [ 0.65, 0.55 ],   #LEFT MOTOR SLOW
  [ 0.75, 0.5 ],    #LEFT MOTOR FASTER
  [ 0.2, 0.2 ],     #BOTH MOTORS FULL BACKWARDS
  [ 1, 0.2 ],       #MOTOR LEFT FULL FORWARD, RIGHT BACKWARDS
  [ 0.5, 0.75 ],    #MOTOR LEFT STOPPED, RIGHT FORWARDS
  [ 0.3, 0.3 ],     #BOTH BACKWARDS 
  [ 0.5, 0.5 ],     #BOTH MOTORS STOPPED
  [ 0.75, 0.75 ],
  [ 1, 0.75 ],
  [ 0.75, 1 ],
  [ 1, 1 ],
  [ 1, 0 ],
  [ 1, 0.75 ],
  [ 0.75, 1 ],
  [ 0.2, 1 ],
  [ 0.65, 0.65],
]

x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train)

In [ ]:
# training data from http://robotics.hobbizine.com/arduinoann.html
x_train = [
  [ 1, 1, 1, 1, 1, 1, 0 ],  # 0
  [ 0, 1, 1, 0, 0, 0, 0 ],  # 1
  [ 1, 1, 0, 1, 1, 0, 1 ],  # 2
  [ 1, 1, 1, 1, 0, 0, 1 ],  # 3
  [ 0, 1, 1, 0, 0, 1, 1 ],  # 4
  [ 1, 0, 1, 1, 0, 1, 1 ],  # 5
  [ 0, 0, 1, 1, 1, 1, 1 ],  # 6
  [ 1, 1, 1, 0, 0, 0, 0 ],  # 7 
  [ 1, 1, 1, 1, 1, 1, 1 ],  # 8
  [ 1, 1, 1, 0, 0, 1, 1 ]   # 9
]

y_train = [
  [ 0, 0, 0, 0 ],  
  [ 0, 0, 0, 1 ], 
  [ 0, 0, 1, 0 ], 
  [ 0, 0, 1, 1 ], 
  [ 0, 1, 0, 0 ], 
  [ 0, 1, 0, 1 ], 
  [ 0, 1, 1, 0 ], 
  [ 0, 1, 1, 1 ], 
  [ 1, 0, 0, 0 ], 
  [ 1, 0, 0, 1 ] 
]

x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train)

In [ ]:
model = Sequential()

IN = x_train.shape[1]
HIDDEN = IN + 1
OUT = y_train.shape[1]

model.add(Dense(units=HIDDEN, activation='sigmoid', input_dim=IN))
model.add(Dense(units=OUT, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer=optimizers.SGD(lr=0.4))

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=1, verbose=False, callbacks=[plot_losses])

In [ ]:
preds = model.predict(x_train).tolist()
for i, pred in enumerate(preds):
    print('model:\t', np.round(pred,2))
    print('label:\t', y_train[i])
    print(' ')

In [ ]:
def to_str(name, W):
    s = str(W.tolist()).replace('[', '{').replace(']', '}')
    return 'float '+name+'['+str(W.shape[0])+']['+str(W.shape[1])+'] = ' + s + ';'

In [ ]:
# output weights as strings you can copy into arduino sketch
W1 = np.vstack((K.eval(model.weights[0]), K.eval(model.weights[1])))
W2 = np.vstack((K.eval(model.weights[2]), K.eval(model.weights[3])))

print('// replace these two lines in arduino code:')
print('// float HiddenWeights[InputNodes+1][HiddenNodes]')
print('// float OutputWeights[HiddenNodes+1][OutputNodes]')
print('// With trained weights below.')
print('\n')
print('// You also have to disable the initial weight randomization')
print('// you can find it under this section in the arduino code:')
print('//* Initialize HiddenWeights and ChangeHiddenWeights ')
print('\n')
print(to_str('HiddenWeights', W1))
print(to_str('OutputWeights', W2))